In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymysql

## 종목, 종목코드 크롤링

In [4]:
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

df = pd.read_html(url, header=0)[0]
df

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
1,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
2,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
3,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
4,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
...,...,...,...,...,...,...,...,...,...
2455,툴젠,199800,자연과학 및 공학 연구개발업,"유전자가위 (유전체 교정 도구) 및 이를 이용한 유전자세포치료제, 고부가가치 가축,...",2014-06-25,12월,"김영호, 이병화(각자 대표이사)",http://www.toolgen.com,서울특별시
2456,틸론,217880,소프트웨어 개발 및 공급업,"Cloud Solution (D,A,E,Rstation)",2015-05-28,12월,최백준,http://www.tilon.co.kr,서울특별시
2457,플럼라인생명과학,222670,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피",2015-07-28,12월,김앤토니경태,http://www.plsdna.com,서울특별시
2458,한국미라클피플사,331660,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등",2019-10-28,12월,이호경,http://www.kmpc.co.kr,경기도


In [5]:
df = df[['회사명', '종목코드']]
df

,회사명,종목코드
0,DRB동일,4840
1,DSR,155660
2,GS,78930
3,GS글로벌,1250
4,HDC현대산업개발,294870
...,...,...
2455,툴젠,199800
2456,틸론,217880
2457,플럼라인생명과학,222670
2458,한국미라클피플사,331660


In [6]:
df['종목코드'] = df['종목코드'].map('{:06d}'.format)

df

<ipython-input-6-e29a63d8a769>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['종목코드'] = df['종목코드'].map('{:06d}'.format)


,회사명,종목코드
0,DRB동일,004840
1,DSR,155660
2,GS,078930
3,GS글로벌,001250
4,HDC현대산업개발,294870
...,...,...
2455,툴젠,199800
2456,틸론,217880
2457,플럼라인생명과학,222670
2458,한국미라클피플사,331660


In [7]:
df['종목코드']

0       004840
1       155660
2       078930
3       001250
4       294870
         ...  
2455    199800
2456    217880
2457    222670
2458    331660
2459    212310
Name: 종목코드, Length: 2460, dtype: object

## 종목. 종목코드 넣기 (이미 넣어져있어 안들어감)

In [9]:
# 전역변수 선언부
conn, cur = None, None

# 메인코드 - 설명
conn = pymysql.connect(host='localhost', user='root', password='1111',
                     db = 'investor', charset ='utf8')

cur = conn.cursor()

# 메인코드
cur.execute('Create table if not exists company_info (id varchar(20), id_code varchar(10), primary key(id_code))')

for i in range(len(df)):
    sql = "INSERT INTO company_info (id, id_code) VALUES (%s, %s)"
    val = (df['회사명'][i], df['종목코드'][i])
    cur.execute(sql, val)   
    
conn.commit()
conn.close()

OperationalError: (1054, "Unknown column 'id' in 'field list'")

In [10]:
len(df)

2458

## db에서 종목코드 뽑아오기

In [9]:
conn = pymysql.connect(host='localhost', user='root', password='1111',
                     db = 'investor', charset ='utf8')

cur = conn.cursor()

cur.execute('select id_code from company_info')

com_list = []
while(True):
    row = cur.fetchone()
    if row == None:
        break
    com_list.append(row)

com_list

[('000020',),
 ('000040',),
 ('000050',),
 ('000060',),
 ('000070',),
 ('000080',),
 ('000100',),
 ('000120',),
 ('000140',),
 ('000150',),
 ('000180',),
 ('000210',),
 ('000220',),
 ('000230',),
 ('000240',),
 ('000250',),
 ('000270',),
 ('000300',),
 ('000320',),
 ('000370',),
 ('000390',),
 ('000400',),
 ('000430',),
 ('000440',),
 ('000480',),
 ('000490',),
 ('000500',),
 ('000520',),
 ('000540',),
 ('000590',),
 ('000640',),
 ('000650',),
 ('000660',),
 ('000670',),
 ('000680',),
 ('000700',),
 ('000720',),
 ('000760',),
 ('000810',),
 ('000850',),
 ('000860',),
 ('000880',),
 ('000890',),
 ('000910',),
 ('000950',),
 ('000970',),
 ('000990',),
 ('001000',),
 ('001020',),
 ('001040',),
 ('001060',),
 ('001070',),
 ('001080',),
 ('001120',),
 ('001130',),
 ('001140',),
 ('001200',),
 ('001210',),
 ('001230',),
 ('001250',),
 ('001260',),
 ('001270',),
 ('001290',),
 ('001340',),
 ('001360',),
 ('001380',),
 ('001390',),
 ('001420',),
 ('001430',),
 ('001440',),
 ('001450',),
 ('001

In [10]:
com_list[1][0]

'000040'

## lastpage 뽑기

In [13]:
url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[1][0]
res = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(res.content, 'html.parser')
print(soup)
data = soup.select_one('td.pgRR')
data.a['href'].split('=')[2]


<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language=

'634'

### 코드별 종목 뽑기

In [11]:
url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[1][0]
response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
data = pd.read_html(response_page)[0]
data = data.dropna()
data = data.reset_index(drop=True)
data

,날짜,종가,전일비,시가,고가,저가,거래량
0,2021.09.17,1050.0,20.0,1080.0,1080.0,1015.0,262580.0
1,2021.09.16,1070.0,5.0,1085.0,1085.0,1065.0,207791.0
2,2021.09.15,1075.0,15.0,1090.0,1100.0,1075.0,284072.0
3,2021.09.14,1090.0,5.0,1095.0,1110.0,1080.0,353381.0
4,2021.09.13,1095.0,5.0,1130.0,1130.0,1085.0,303989.0
5,2021.09.10,1100.0,10.0,1080.0,1125.0,1075.0,472803.0
6,2021.09.09,1090.0,25.0,1120.0,1120.0,1090.0,366886.0
7,2021.09.08,1115.0,5.0,1120.0,1165.0,1095.0,650881.0
8,2021.09.07,1110.0,10.0,1125.0,1135.0,1105.0,226623.0
9,2021.09.06,1120.0,5.0,1130.0,1130.0,1105.0,351201.0


In [12]:
for i in range(len(com_list)):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[i][0]
    response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
    data = pd.read_html(response_page)[0]
    data = data.dropna()
    data = data.reset_index(drop=True)
    data

KeyboardInterrupt: 

## naver를 db에 넣어보기

In [13]:
url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[1][0]
response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
data = pd.read_html(response_page)[0]
data = data.dropna()
data = data.reset_index(drop=True)
data = data.rename(columns = {'날짜': 'date','종가':'close','시가':'open','고가':'heigh','저가':'low','거래량':'volume','전일비':'diff'})
data

,date,close,diff,open,heigh,low,volume
0,2021.09.17,1050.0,20.0,1080.0,1080.0,1015.0,262580.0
1,2021.09.16,1070.0,5.0,1085.0,1085.0,1065.0,207791.0
2,2021.09.15,1075.0,15.0,1090.0,1100.0,1075.0,284072.0
3,2021.09.14,1090.0,5.0,1095.0,1110.0,1080.0,353381.0
4,2021.09.13,1095.0,5.0,1130.0,1130.0,1085.0,303989.0
5,2021.09.10,1100.0,10.0,1080.0,1125.0,1075.0,472803.0
6,2021.09.09,1090.0,25.0,1120.0,1120.0,1090.0,366886.0
7,2021.09.08,1115.0,5.0,1120.0,1165.0,1095.0,650881.0
8,2021.09.07,1110.0,10.0,1125.0,1135.0,1105.0,226623.0
9,2021.09.06,1120.0,5.0,1130.0,1130.0,1105.0,351201.0


In [24]:
type(data['diff'][1])

numpy.float64

In [ ]:
###############################################
#여기서 에러가 나옵니다.!..

# 전역변수 선언부
conn, cur = None, None

# 메인코드 - 설명
conn = pymysql.connect(host='localhost', user='root', password='1111',
                     db = 'investor', charset ='utf8')

cur = conn.cursor()

# 메인코드
sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
cur.execute(sql)

#f"replace into company_info (code, company, last_update) values('{code}', '{company}', '{today}')"

for i in range(len(data)):
    code = com_list[1][0]
    date = data.date[i]
    open = data.open[i]
    heigh = data.heigh[i]
    low = data.low[i]
    close = data.close[i]
    diff = data['diff'][i]
    volume = data.volume[i]
    
    sql = f"INSERT INTO daily_price (code, date, open, high, low, close, diff, volume) VALUES ('{code}','{date}','{open}','{heigh}','{low}','{close}','{diff}','{volume}')"
    cur.execute(sql) 
    
    
    
conn.commit()
conn.close()

In [1]:
url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[1][0]
response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
data = pd.read_html(response_page)[0]
data = data.dropna()
data = data.reset_index(drop=True)
data = data.rename(columns = {'날짜': 'date','종가':'close','시가':'open','고가':'heigh','저가':'low','거래량':'volume','전일비':'diff'})
data



NameError: name 'com_list' is not defined

In [17]:
#### 합본 조심해서 건들이시오
sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
cur.execute(sql)
conn.commit()



for i in range(len(com_list)):                                                      #각 종목별로 마지막 페이지 뽑기
    
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[i][0]
    res = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.select_one('td.pgRR')
    if data != None:
        s = data.a['href'].split('=')
        last_page = s[-1]
        print(last_page)
    else:
        last_page = 1
        print(1)
        
    if int(last_page) <5:                                                                                # 마지막 페이지에 따른 페이지 크롤링
        for j in range(int(last_page)):
            url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'%(com_list[i][0], j+1)
            response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
            data = pd.read_html(response_page)[0]
            data = data.dropna()
            data = data.reset_index(drop=True)
            data = data.rename(columns = {'날짜': 'date','종가':'close','시가':'open','고가':'heigh',
                                          '저가':'low','거래량':'volume','전일비':'diff'})
            data

            for idx in range(len(data)):
                code = com_list[i][0]
                date = data.date[idx]
                open = data.open[idx]
                heigh = data.heigh[idx]
                low = data.low[idx]
                close = data.close[idx]
                diff = data['diff'][idx]
                volume = data.volume[idx]
                sql = f"INSERT INTO daily_price (code, date, open, high, low, close, diff, volume) VALUES ('{code}','{date}','{open}','{heigh}','{low}','{close}','{diff}','{volume}')"
                cur.execute(sql) 

    elif int(last_page) >= 5:
        for k in range(2):
            url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'%(com_list[i][0], k+1)
            response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
            data = pd.read_html(response_page)[0]
            data = data.dropna()
            data = data.reset_index(drop=True)
            data = data.rename(columns = {'날짜': 'date','종가':'close','시가':'open','고가':'heigh','저가':'low','거래량':'volume','전일비':'diff'})
            data

            for idx in range(len(data)):
                code = com_list[i][0]
                date = data.date[idx]
                open = data.open[idx]
                heigh = data.heigh[idx]
                low = data.low[idx]
                close = data.close[idx]
                diff = data['diff'][idx]
                volume = data.volume[idx]
                sql = f"INSERT INTO daily_price (code, date, open, high, low, close, diff, volume) VALUES ('{code}','{date}','{open}','{heigh}','{low}','{close}','{diff}','{volume}')"
                cur.execute(sql) 
    
    conn.commit()
    
conn.close()





633
634
633
633
634
295
634
634
634
634
634
634
634
634
634
518
634
634
634
634
634
634
634
577
634
634
634
634
634
634
634
634
619
634
634
634
634
634
634
539
634
634
634
634
634
634
634
576
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
526
634
391
634
634
634
634
634
634
634
634
354
634
634
533
634
576
634
634
634
634
634
634
634
274
634
634
634
634
577
634
634
577
634
634
634
634
634
634
634
634
634
634
634
634
634
577
148
634
634
634
634
634
634
112
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
534
634
631
634
577
634
634
634
634
576
634
104
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
492
634
634
634
634
576
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
634
135
634
634
634
634
576
176
634
634
634
634
634
577
634
634
634
634
402
634
634
634
634
634
634
634
634
634
634
634
546
634
634
534
634
620
634
634
634
634
634
634
634
634
634
577
634
634
577


101
108
82
99
3
89
100
94
116
105
30
16
70
68
99
100
70
86
102
99
110
102
97
102
98
110
90
106
110
110
97
20
106
29
109
100
108
108
108
108
105
90
108
70
106
107
71
92
99
105
101
103
69
104
105
2
103
48
46
104
93
94
102
1
39
77
80
9
87
98
16
96
65
45
96
61
95
96
93
93
94
93
94
53
81
91
92
72
93
20
52
78
84
44
70
73
55
80
13
69
78
77
45
30
68
72
70
71
22
74
76
75
25
74
26
74
57
77
26
39
10
40
81
46
79
31
79
79
79
79
32
68
29
68
21
74
80
64
80
68
70
54
48
77
23
69
13
77
77
76
68
7
29
23
57
46
74
72
68
69
68
69
58
62
2
57
46
68
56
68
68
68
68
68
52
43
56
62
54
68
13
3
65
62
50
60
59
53
47
47
20
53
52
54
45
48
53
48
20
25
6
59
59
50
58
46
16
46
47
18
15
44
58
31
55
56
56
58
3
56
25
31
43
55
53
52
35
51
31
47
21
49
49
47
48
27
49
27
29
47
47
47
12
30
14
19
45
45
45
45
48
48
44
45
45
28
45
15
22
43
44
38
44
43
21
15
41
43
41
35
29
40
42
35
23
37
13
26
25
19
29
16
21
24
23
27
28
34
28
31
11
30
9
16
20
24
5
20
19
29
32
34
27
33
20
25
19
14
27
9
28
19
6
28
25
13
10
9
9
16
25
4
19
19
16
3
20
13


In [20]:
for i in range(10):
    try:
        if i != 8:
            print(i)
    except:
        pass
        
    print(i*10)

0
0
1
10
2
20
3
30
4
40
5
50
6
60
7
70
80
9
90


In [16]:
# krx 종목코드 크롤링
def krx_code():
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

    df = pd.read_html(url, header=0)[0]
    df = df[['회사명', '종목코드']]
    df['종목코드'] = df['종목코드'].map('{:06d}'.format)
    print(df)
    return df

In [17]:
a = krx_code()

            회사명    종목코드
0         DRB동일  004840
1           DSR  155660
2            GS  078930
3         GS글로벌  001250
4     HDC현대산업개발  294870
...         ...     ...
2455         툴젠  199800
2456         틸론  217880
2457   플럼라인생명과학  222670
2458   한국미라클피플사  331660
2459        휴벡셀  212310

[2460 rows x 2 columns]


In [19]:
# krx db에 넣기 
def insert_krx(df):
    # 메인코드 - 설명
    conn = pymysql.connect(host='localhost', user='root', password='1111',
                         db = 'investor', charset ='utf8')
    cur = conn.cursor()

    # 메인코드
    cur.execute('Create table if not exists company_info (id varchar(20), id_code varchar(10), primary key(id_code))')

    for i in range(len(df)):
        sql = "replace INTO company_info (id, id_code) VALUES (%s, %s)"
        val = (df['회사명'][i], df['종목코드'][i])
        cur.execute(sql, val)   

    conn.commit()
    conn.close()

In [20]:
data = krx_code()
insert_krx(data)

            회사명    종목코드
0         DRB동일  004840
1           DSR  155660
2            GS  078930
3         GS글로벌  001250
4     HDC현대산업개발  294870
...         ...     ...
2455         툴젠  199800
2456         틸론  217880
2457   플럼라인생명과학  222670
2458   한국미라클피플사  331660
2459        휴벡셀  212310

[2460 rows x 2 columns]


OperationalError: (1054, "Unknown column 'id' in 'field list'")

In [21]:
# db에서 조목코드 뽑아오기
conn = pymysql.connect(host='localhost', user='root', password='1111',
                     db = 'investor', charset ='utf8')

cur = conn.cursor()

cur.execute('select code from company_info')

com_list = []
while(True):
    row = cur.fetchone()
    if row == None:
        break
    com_list.append(row)

com_list

[('000020',),
 ('000040',),
 ('000050',),
 ('000060',),
 ('000070',),
 ('000080',),
 ('000100',),
 ('000120',),
 ('000140',),
 ('000150',),
 ('000180',),
 ('000210',),
 ('000220',),
 ('000230',),
 ('000240',),
 ('000250',),
 ('000270',),
 ('000300',),
 ('000320',),
 ('000370',),
 ('000390',),
 ('000400',),
 ('000430',),
 ('000440',),
 ('000480',),
 ('000490',),
 ('000500',),
 ('000520',),
 ('000540',),
 ('000590',),
 ('000640',),
 ('000650',),
 ('000660',),
 ('000670',),
 ('000680',),
 ('000700',),
 ('000720',),
 ('000760',),
 ('000810',),
 ('000850',),
 ('000860',),
 ('000880',),
 ('000890',),
 ('000910',),
 ('000950',),
 ('000970',),
 ('000990',),
 ('001000',),
 ('001020',),
 ('001040',),
 ('001060',),
 ('001070',),
 ('001080',),
 ('001120',),
 ('001130',),
 ('001140',),
 ('001200',),
 ('001210',),
 ('001230',),
 ('001250',),
 ('001260',),
 ('001270',),
 ('001290',),
 ('001340',),
 ('001360',),
 ('001380',),
 ('001390',),
 ('001420',),
 ('001430',),
 ('001440',),
 ('001450',),
 ('001

In [29]:
# 함수 만들기1  인서트하는 부분

def insert_db(data):
    for idx in range(len(data)):
        code = com_list[i][0]
        date = data.date[idx]
        open = data.open[idx]
        heigh = data.heigh[idx]
        low = data.low[idx]
        close = data.close[idx]
        diff = data['diff'][idx]
        volume = data.volume[idx]
        sql = f"replace INTO daily_price (code, date, open, high, low, close, diff, volume) VALUES ('{code}','{date}','{open}','{heigh}','{low}','{close}','{diff}','{volume}')"
        cur.execute(sql) 

In [30]:
# 함수 만들기! 데이터 정제하기

def make_db(j):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'%(com_list[i][0], j+1)
    response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
    data = pd.read_html(response_page)[0]
    data = data.dropna()
    data = data.reset_index(drop=True)
    data = data.rename(columns = {'날짜': 'date','종가':'close','시가':'open','고가':'heigh','저가':'low','거래량':'volume','전일비':'diff'})
    return data


In [31]:
#### 총합본 조심해서 건들이시오
# 네이버에서 주식종목 크롤링해서 db에 넣기

conn = pymysql.connect(host='localhost', user='root', password='1111',
                         db = 'investor', charset ='utf8')
cur = conn.cursor()

sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
cur.execute(sql)
conn.commit()



for i in range(len(com_list)):                                                      #각 종목별로 마지막 페이지 뽑기
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[i][0]
    res = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.select_one('td.pgRR')
    if data != None:
        s = data.a['href'].split('=')
        last_page = s[-1]
        last_page = 3
        print(last_page)
    else:
        last_page = 1
        print(last_page)
        
    for j in range(int(last_page)):
        data  = make_db(j)
        insert_db(data)
    
    conn.commit()
    
conn.close()


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [19]:
com_list[-1][0]

'950220'

In [ ]:
950220

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymysql


# krx 종목코드 크롤링
def krx_code():
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

    df = pd.read_html(url, header=0)[0]
    df = df[['회사명', '종목코드']]
    df['종목코드'] = df['종목코드'].map('{:06d}'.format)
    print(df)
    return df


# krx db에 넣기 
def insert_krx(df):
    # 메인코드 - 설명
    conn = pymysql.connect(host='localhost', user='root', password='1111',
                         db = 'investor', charset ='utf8')
    cur = conn.cursor()

    # 메인코드
    cur.execute('Create table if not exists company_info (id varchar(20), id_code varchar(10), primary key(id_code))')

    for i in range(len(df)):
        sql = "replace INTO company_info (id, id_code) VALUES (%s, %s)"
        val = (df['회사명'][i], df['종목코드'][i])
        cur.execute(sql, val)   

    conn.commit()
    conn.close()


# db에서 조목코드 뽑아오기
def select_krx():
    conn = pymysql.connect(host='localhost', user='root', password='1111',
                        db = 'investor', charset ='utf8')
    cur = conn.cursor()
    cur.execute('select id_code from company_info')
    com_list = []
    while(True):
        row = cur.fetchone()
        if row == None:
            break
        com_list.append(row)
    com_list

    return com_list


# 함수 만들기1  인서트하는 부분

def insert_db(data):
    for idx in range(len(data)):
        code = com_list[i][0]
        date = data.date[idx]
        open = data.open[idx]
        heigh = data.heigh[idx]
        low = data.low[idx]
        close = data.close[idx]
        diff = data['diff'][idx]
        volume = data.volume[idx]
        sql = f"replace INTO daily_price (code, date, open, high, low, close, diff, volume) VALUES ('{code}','{date}','{open}','{heigh}','{low}','{close}','{diff}','{volume}')"
        cur.execute(sql) 


# 함수 만들기! 데이터 정제하기

def make_db(j):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'%(com_list[i][0], j+1)
    response_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
    data = pd.read_html(response_page)[0]
    data = data.dropna()
    data = data.reset_index(drop=True)
    data = data.rename(columns = {'날짜': 'date','종가':'close','시가':'open','고가':'heigh','저가':'low','거래량':'volume','전일비':'diff'})
    return data


#### 총합본 조심해서 건들이시오
# 네이버에서 주식종목 크롤링해서 db에 넣기

krx_data = krx_code()
insert_krx(krx_data)
com_list = select_krx()


conn = pymysql.connect(host='localhost', user='root', password='1111',
                         db = 'investor', charset ='utf8')
cur = conn.cursor()

sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
cur.execute(sql)
conn.commit()



for i in range(len(com_list)):                                                      #각 종목별로 마지막 페이지 뽑기
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s'%com_list[i][0]
    res = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.select_one('td.pgRR')
    if data != None:
        s = data.a['href'].split('=')
        last_page = s[-1]
        last_page = 3
        print(last_page)
    else:
        last_page = 1
        print(last_page)
        
    for j in range(int(last_page)):
        data  = make_db(j)
        insert_db(data)
    
    conn.commit()
    
conn.close()


            회사명    종목코드
0         DRB동일  004840
1           DSR  155660
2            GS  078930
3         GS글로벌  001250
4     HDC현대산업개발  294870
...         ...     ...
2455         툴젠  199800
2456         틸론  217880
2457   플럼라인생명과학  222670
2458   한국미라클피플사  331660
2459        휴벡셀  212310

[2460 rows x 2 columns]
3
3
3
3
3
3
3
3
3
3
3


KeyboardInterrupt: 

In [ ]:
http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13

In [ ]:
http://finance.naver.com/item/sise_day.nhn?code=